In [ ]:
import pickle
import time
start_time = time.time()
import numpy as np
import os
import SimpleITK as sitk
import json
import ProstateLesionDetectionUtils
import random
import plotly.express as px
import matplotlib.pyplot as plt
from scipy import ndimage
import math
import tensorflow_addons as tfa
import tensorflow as tf
with open(r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\UC1_Nifti_Datasets_Paths\Datasets.json", 'r') as f:
    datasets = json.load(f)
p158 = r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PROSTATE158\DATASETPICKLE\Nifti_files"
picai = r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PICAI_DATA\Nifti_files"
prostatenet = r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PROSTATENET_UC1_LESIONS\nnUNet_raw\Dataset011_ProstateNet"
with open(r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PROSTATENET_UC1_LESIONS\nnUNet_raw\Dataset011_ProstateNet\folds.json", 'r') as f:
    ValidationFolds = json.load(f)
    


In [ ]:
loader = ProstateLesionDetectionUtils.DatasetsLoaders.Nifti_Loader.LoadSitkImages(ptdc = datasets["ProstateNet"], folder = prostatenet)
patients = loader.load_sitkobj()
patients_test = {key:patients[key] for key in patients.keys() if key in ValidationFolds["Fold 0"]}

In [ ]:
eval_params = {
    "MODEL": transunet,
    "EVALUATION_DATA": patients_test,# Needs to be a dictionary {PatName:{T2:arr, ADC:arr, DWI:arr, "Lesion":arr}}
    "SAVE_FOLDER":os.path.join(*path_parent) ,
}

base_path = r"E:\DIMITRIS_DEEP_LEARNING\RESULTS\PROCANCER_UC1_RESULTS"
model  = "Trans_Unet"
fold = "fold_0"
weights = "Final_1024.tf"
paths = [base_path, model, fold, weights] # chains the paths
path_parent = [base_path, model, fold]

transunet = tf.keras.models.load_model(os.path.join(*paths),compile = False)

eval = Evaluation(eval_params)
eval.predict()
eval.calculate_detorient_metric()
eval.save_gt()
eval.save_predictions()
eval.save_predictions_binary()
metrics = eval.get_metrics()
preds = eval.get_predictions()